### Step 1: Install CANYA

Google Colab 환경에서 Conda 없이 바로 CANYA를 설치합니다. 필요한 종속성(`tensorflow`, `numpy`, `pandas`)은 Colab에 이미 설치되어 있으므로 별도 설치가 필요하지 않습니다.

In [ ]:
!python -m pip install --no-cache-dir https://github.com/lehner-lab/canya/tarball/master

### Step 2: Prepare Input File

`canya_input.txt`라는 입력 파일을 생성합니다. 형식은 CANYA에서 요구하는 형식에 맞게 구성해야 합니다 (예: 탭 구분 서열 리스트).

In [ ]:
with open("canya_input.txt", "w") as f:
    f.write("ID\tSequence\n")
    f.write("Seq1\tCASSIRSSYEQYF\n")
    f.write("Seq2\tCASSLGQGAETLYF\n")

### Step 3: Run CANYA (Basic Mode)

In [ ]:
!canya --input canya_input.txt --output basic_out.txt

### Step 4: Run CANYA with Summary Mode (`--summarize mean`)

In [ ]:
!canya --input canya_input.txt --output summarized_out.txt --summarize mean

### Step 5: Run CANYA in Ensemble Mode (Robust Prediction)

In [ ]:
!canya --input canya_input.txt --output ensemble_out.txt --summarize mean --mode ensemble

### Step 6: Visualize Results

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

try:
    df = pd.read_csv("ensemble_out.txt", sep='\t')
    df_sorted = df.sort_values('Mean_Score', ascending=False).reset_index(drop=True)

    plt.figure(figsize=(10, 6))
    sns.barplot(x='ID', y='Mean_Score', data=df_sorted, palette='viridis', edgecolor='black')
    plt.errorbar(x=df_sorted.index, y=df_sorted['Mean_Score'], yerr=df_sorted['Std_Dev_Score'], fmt='none', c='black', capsize=5)
    plt.title('CANYA Nucleation Score (Ensemble Mode)', fontsize=16)
    plt.xlabel('Sequence ID')
    plt.ylabel('Mean ± Std Dev')
    plt.ylim(0, 1)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print("Error reading results:", e)